<a href="https://colab.research.google.com/github/M3liss/BA-Project---1st-draft/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import xml.etree.cElementTree as et
import PIL
import os
import random

In [2]:
! git clone https://github.com/M3liss/BA-Project---1st-draft.git

Cloning into 'BA-Project---1st-draft'...
remote: Enumerating objects: 430, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 430 (delta 11), reused 2 (delta 0), pack-reused 406
Receiving objects: 100% (430/430), 12.24 MiB | 40.44 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [3]:
#HORIZONTAL FLIPPING
def hor_flip(img, anno):
    global WIDTH, PATH
    new_name = img.split('/',3)[-1]
    new_name = PATH + "/fli_" + new_name
    new_img = tf.io.read_file(img)
    new_img = tf.image.decode_jpeg(new_img, channels=3)
    new_img = tf.image.flip_left_right(new_img)
    lol = tf.io.encode_jpeg(new_img)
    tf.io.write_file(new_name, lol)
    tree = et.parse(anno)
    root=tree.getroot()
    for i in root.iter('bndbox'):
        xmin = str(i.find('xmin').text)
        xmax = str(i.find('xmax').text)
        xmin = WIDTH - int(xmin)
        xmax = WIDTH - int(xmax)
        for n in i.iter('xmin'):
            n.text = str(xmax)
        for n in i.iter('xmax'):
            n.text = str(xmin)
    tree.write(new_name.replace("jpg", "xml"))

In [4]:
#ROTATION
def rotation(img, anno):
    global WIDTH, HEIGHT, PATH 
    new_name = img.split('/',3)[-1]
    new_name = PATH + "/rot_" + new_name
    new_img = tf.io.read_file(img)
    new_img = tf.image.decode_jpeg(new_img, channels=3)
    new_img = tf.image.rot90(new_img, k=-2)
    lol = tf.io.encode_jpeg(new_img)
    tf.io.write_file(new_name, lol)
    tree = et.parse(anno)
    root=tree.getroot()
    for i in root.iter('bndbox'):
        xmin = str(i.find('xmin').text)
        ymin = str(i.find('ymin').text)
        xmax = str(i.find('xmax').text)
        ymax = str(i.find('ymax').text)
        xmin = WIDTH - int(xmin)
        ymin = HEIGHT - int(ymin)
        xmax = WIDTH - int(xmax)
        ymax = HEIGHT - int(ymax)
        for n in i.iter('xmin'):
            n.text = str(xmax)
        for n in i.iter('ymin'):
            n.text = str(ymax)
        for n in i.iter('xmax'):
            n.text = str(xmin)
        for n in i.iter('ymax'):
            n.text = str(ymin)
    tree.write(new_name.replace("jpg", "xml"))


In [ ]:
from pathlib import Path
#PIL.Image.open(str(PATH))
PATH = "BA-Project---1st-draft/annotations"
WIDTH = 640
HEIGHT = 360
#the code that runs it ALL
files = Path(PATH).glob('*')
rand_operation = [hor_flip, rotation]
names = ["hor_flip", "rotation"]
for file in files:
    file = str(file)
    anno_file = file.replace("jpg", "xml")
    if (file[-3] == "j" and file.find("rot_") == -1 and file.find("fli_") == -1 and os.path.exists(anno_file)):
        op = random.randint(0, len(rand_operation)-1)
        rand_operation[op](file, anno_file)
        print(f"The operation {names[op]} was successfully performed on {file}")